# H264 Video Frame Playback from JSON

This notebook loads H264 video frame data from JSON and plays it back with the original timing to replicate the live streaming experience.

## Process:
1. Load JSON containing frame metadata and base64-encoded H264 data
2. Convert base64 strings to byte arrays (`data_byte_array` field)
3. Decode and display frames one by one with timestamp-based delays

In [ ]:
import json
import base64
import sys
from PyQt5.QtWidgets import QApplication, QFileDialog

# Initialize Qt Application (needed for file dialog)
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)

# Open file picker dialog to select JSON file
json_file_path, _ = QFileDialog.getOpenFileName(
    None,
    "Select Frame Metadata JSON File",
    "2026_02_13",  # Default directory
    "JSON Files (*.json);;All Files (*.*)"
)

# Check if a file was selected
if not json_file_path:
    print("No file selected. Exiting.")
    raise SystemExit("File selection cancelled")

print(f"Selected file: {json_file_path}\n")

# Load the frames metadata JSON file
with open(json_file_path, 'r') as f:
    data = json.load(f)

# Convert base64 data to byte arrays for all frames
print("Converting base64 data to byte arrays...")
for i, frame in enumerate(data['frames']):
    frame['data_byte_array'] = base64.b64decode(frame['data'])

print(f"Conversion complete!\n")

# Display basic information about the loaded data
print(f"Train ID: {data['trainId']}")
print(f"Export Timestamp: {data['exportTimestamp']}")
print(f"Time Range: {data['timeRange']['startTime']} - {data['timeRange']['endTime']}")
print(f"Frame Count: {data['frameCount']}")

## Data Structure

After loading, each frame in `data['frames']` contains:
- `frameId`: Original frame number from source (may have gaps)
- `timestamp`: Unix timestamp in milliseconds
- `latency`: Frame latency in milliseconds
- `size`: Frame size in bytes
- `data`: Base64-encoded H264 frame data (original)
- `data_byte_array`: Decoded byte array (added by Cell 2)

The byte arrays are used to create a playable video buffer.

In [ ]:
import av
import numpy as np
import time
import threading
import queue
from PyQt5.QtWidgets import QApplication, QLabel, QVBoxLayout, QWidget
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtCore import Qt

class VideoDisplayWindow(QWidget):
    """Qt window for displaying video frames"""
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Video Frame Playback")
        
        # Create label for displaying frames
        self.image_label = QLabel()
        self.image_label.setAlignment(Qt.AlignCenter)
        self.image_label.setScaledContents(False)
        
        # Info label for frame metadata
        self.info_label = QLabel()
        self.info_label.setAlignment(Qt.AlignCenter)
        self.info_label.setStyleSheet("font-size: 14px; padding: 10px;")
        
        # Layout
        layout = QVBoxLayout()
        layout.addWidget(self.info_label)
        layout.addWidget(self.image_label)
        self.setLayout(layout)
        
        # Set initial size
        self.resize(800, 600)
        
    def display_frame(self, frame_rgb, frame_id, latency_ms):
        """Update the display with a new frame"""
        height, width, channel = frame_rgb.shape
        bytes_per_line = 3 * width
        
        # Convert numpy array to QImage
        q_image = QImage(frame_rgb.data, width, height, bytes_per_line, QImage.Format_RGB888)
        
        # Convert to QPixmap and display
        pixmap = QPixmap.fromImage(q_image)
        self.image_label.setPixmap(pixmap)
        
        # Update info label
        self.info_label.setText(f"Frame ID: {frame_id} | Latency: {latency_ms} ms")

class FrameDecoder:
    def __init__(self):
        self.codec = av.CodecContext.create('h264', 'r')
        self.prev_timestamp = None
        
        # Queue for passing decoded frames to display thread
        self.frame_queue = queue.Queue(maxsize=30)  # Buffer up to 30 frames
        
        # Threading control
        self.decode_thread = None
        self.stop_event = threading.Event()
        
        # Qt display window
        self.display_window = None
        self.next_frame_time = 0
        
    def decode_frame(self, frame_data):
        """Decode a single frame and put it in the queue"""
        av_packet = av.Packet(frame_data['data_byte_array'])
        try:
            frame = self.codec.decode(av_packet)
            current_timestamp = frame_data['timestamp']

            # Calculate delay based on timestamp difference
            delay_ms = 0
            if self.prev_timestamp is not None:
                delay_ms = current_timestamp - self.prev_timestamp
            
            self.prev_timestamp = current_timestamp

            # Prepare all the data to be displayed
            frame_id = frame_data['frameId']
            frame_rgb = frame[0].to_ndarray(format='rgb24')
            latency_ms = frame_data['latency']

            # Put decoded frame data in queue for display thread
            self.frame_queue.put({
                'frame_rgb': frame_rgb,
                'frame_id': frame_id,
                'latency_ms': latency_ms,
                'delay_ms': delay_ms
            })

        except Exception as e:
            print(f"Error decoding frame {frame_data['frameId']}: {e}")

    def decode_worker(self, frames_data):
        """Worker function for decode thread"""
        print("Decode thread started")
        for frame_data in frames_data:
            if self.stop_event.is_set():
                break
            self.decode_frame(frame_data)
        
        # Signal end of frames
        self.frame_queue.put(None)
        print("Decode thread finished")

    def start_playback(self, frames_data):
        """Start decode thread and display loop"""
        self.stop_event.clear()
        
        # Create Qt display window (in main thread)
        self.display_window = VideoDisplayWindow()
        self.display_window.show()
        
        # Start decode thread
        self.decode_thread = threading.Thread(target=self.decode_worker, args=(frames_data,))
        self.decode_thread.start()
        
        print("Playback started - processing frames...")
        
        # Main display loop - processes Qt events and displays frames
        self.next_frame_time = time.time() * 1000
        
        while not self.stop_event.is_set():
            # Process Qt events to keep window responsive
            QApplication.processEvents()
            
            # Check if window was closed
            if not self.display_window.isVisible():
                print("Window closed by user")
                self.stop_playback()
                break
            
            current_time = time.time() * 1000  # milliseconds
            
            # Check if it's time to display next frame
            if current_time >= self.next_frame_time:
                try:
                    # Try to get a frame from queue (non-blocking)
                    frame_info = self.frame_queue.get_nowait()
                    
                    # None signals end of frames
                    if frame_info is None:
                        print("All frames displayed!")
                        break
                    
                    # Display the frame
                    self.display_window.display_frame(
                        frame_info['frame_rgb'], 
                        frame_info['frame_id'], 
                        frame_info['latency_ms']
                    )
                    
                    # Schedule next frame based on delay
                    self.next_frame_time = current_time + frame_info['delay_ms']
                    
                except queue.Empty:
                    # No frame available yet, just continue
                    pass
            
            # Small sleep to avoid busy waiting
            time.sleep(0.001)
        
        # Cleanup
        self.stop_event.set()
        if self.decode_thread and self.decode_thread.is_alive():
            self.decode_thread.join(timeout=2)
        
        if self.display_window and self.display_window.isVisible():
            self.display_window.close()
        
        print("Playback completed")


In [ ]:
# Create decoder instance and start playback
decoder = FrameDecoder()

# This will block until playback completes or window is closed
decoder.start_playback(data['frames'])

In [ ]:
# If you need to force stop playback, run this cell
# Or simply close the playback window
decoder.stop_event.set()